In [ ]:
import os
import pandas as pd
import sqlite3
import numpy as np
import yaml
import pickle
import time
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from scipy import stats
import datetime as dt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
def make_config(config_write_path):
    """
    config_write_path : folder path to write config file
    """
    config = {}
    config['id_columns'] = ['BatchNo', 'starttime', 'season', 'Month', 'endtime', 'BrandID', 'BatchID']
    
    config['imp_columns'] = ['AC BASE DUSTING MTR CURRENT', 'CG1 MTR TT', 'CG2 AHU MTR CURRENT', 'CG2 AHU RH', 'CG2 AHU TT',
                             'CG2 DC MTR CURRENT', 'CG2 MTR PT', 'CG2 MTR TT', 'COURSE SODA SP', 'DC FINES SP', 'DC FINES WT',
                             'FBC INLET TT', 'FBC1 BLW PT', 'FBC1 SF10202A CURRENT', 'FBC1 SF10202B CURRENT',
                             'HLAS HOT TANK TEMP', 'MT 3RD FLOOR', 'MT BH', 'NI DOSING TANK TEMP', 'PADDELMIXER HLAS SP',
                             'PADDELMIXER NI SP', 'PADDELMIXER RV SP', 'POSTMIX AC BASE SP', 'POSTMIX CLAY SP',
                             'POSTMIXING CURRENT', 'PREMIX AC BASE SP', 'PREMIX SODA SP', 'PREMIX SULPHATE SP', 'RM FINES SP',
                             'RV BASE DOSING TANK TEMP', 'TT 3RD FLOOR', 'TT BH', 'ST GRINDER BRG TT1', 'ST GRINDER BRG TT2',
                             'ST GRINDER BRG TT3', 'ST GRINDER BRG TT4']
    
    config['train_data_split'] = [0.7]
    config['daily_model_list'] = []
    write_config(config, os.path.join(config_write_path, 'config.txt'))
    return

In [ ]:
def write_config(data_dict, config_file_path):
    def cross(dd):
        key_list = list(set(list(dd.keys())) - set(['id_columns', 'imp_columns', 'train_data_split']))
        for i in key_list:
            l = dd[i]
            if len(l)==0:
                continue;
            if len(l)==1:
                try:
                    dd[i] = [os.path.abspath(l[0])]
                except:
                    continue;
            else:
                try:
                    dd[i] = [os.path.abspath(l[j]) for j in range(len(l))]
                except:
                    continue;
        return dd
    data_dict = cross(data_dict)
    t = open(config_file_path, 'w')
    for ind, key in enumerate(data_dict):
        if ind==0:
            t.write("{\n")
            t.write("{}:{},\n".format(key, data_dict[key]))#str(key), str(data_dict[key])
        elif ind==(len(data_dict)-1):
            t.write("{}:{}\n".format(key, data_dict[key]))
            t.write("}")
        else:
            t.write("{}:{},\n".format(key, data_dict[key]))
    t.close()

def read_config(config_file_path):
    rt = open(config_file_path, 'r')
    lines = rt.readlines()
    sk = ''.join(map(str, [i.strip() for i in lines]))
    config_dict = yaml.load(sk)
    rt.close()
    return config_dict

In [ ]:
def consolidate_Agglomeration_Data(excel_folder_path, write_path, config_path):
    """
    excel_folder_path : folder path of xlsx files
    write_path      : write file path
    config_path : full path of config file (.../.../xxxxx.txt)
    
    """
    fil = [i for i in os.listdir(excel_folder_path) if i.endswith('.xlsx')]
    if len(fil)==0:
        fil = [i for i in os.listdir(excel_folder_path) if i.endswith('.csv')]
    
    ind = [int(fil[i].split('-')[0]) for i in range(len(fil))]
    
    files = [l for _,l in sorted(zip(ind, fil))]
    
    for i in range(len(files)):
        if files[0].endswith('.xlsx'):
            f = pd.read_excel(os.path.join(excel_folder_path, files[i]), sheet_name='Agglo')
        else:
            f = pd.read_csv(os.path.join(excel_folder_path, files[i]), sheet_name='Agglo')
        
        if i==0:
            df = pd.DataFrame(columns=list(f.columns))
        
        df = pd.concat([df, f], ignore_index=True, axis=0)
    
    name = 'consolidated_Agglomeration_Data2'+ '_' + str(int(time.time())) + '.csv'
    df.to_csv(os.path.join(write_path, name))
    config = read_config(config_path)
    config['consolidated_agglo_data_path'] = [os.path.join(write_path, name)]
    write_config(config, config_path)
    return df


def consolidate_centreline_Data(excel_file_path, write_path, config_path):
    """
    excel_file_path : path of excel [/.../..xlsx]
    write_path      : write file path
    config_path : full path of config file (.../.../xxxxx.txt)
    
    """
    df = pd.concat(pd.read_excel(excel_file_path, sheet_name=None), ignore_index=True)
    
    name = 'consolidated_centreline_Data2' + '_' + str(int(time.time())) + '.csv'
    
    df.to_csv(os.path.join(write_path, name))
    config = read_config(config_path)
    config['consolidated_centreline_data_path'] = [os.path.join(write_path, name)]
    write_config(config, config_path)
    return df


In [ ]:
def merge_data(agglo_table, centreline_table, database_path, config_path, mode='pred'):
    """
    config_path : full path of config file (.../.../xxxxx.txt)
    mode        : pred or train
    
    """
    config = read_config(config_path)
    # if mode=='train':
    #     agglo_path = config['consolidated_agglo_data_path'][0]
    #     centre_line_path = config['consolidated_centreline_data_path'][0]
    #     write_path = os.getcwd()#os.path.dirname(centre_line_path)
        
    # elif len(config['daily_model_list'])>0:
    #     folder_path = os.path.abspath(config['daily_model_list'][-1])
    #     agglo_path = os.path.join(folder_path, 'agglo_data', 'agglo.xlsx')
    #     centre_line_path = os.path.join(folder_path, 'centreline_data', 'centreline.xlsx')
    #     write_path = str(folder_path)
    # else:
    #     today_date = dt.datetime.today().strftime ("%Y-%m-%d")
    #     folder_path = os.path.join(os.path.dirname(config['results_params_csv_path'][0]), str(today_date))
    #     agglo_path = os.path.join(folder_path, 'agglo_data', 'agglo.xlsx')
    #     centre_line_path = os.path.join(folder_path, 'centreline_data', 'centreline.xlsx')
    #     write_path = str(folder_path)
        
    agglo = table_to_df(agglo_table)
    centerline = table_to_df(centreline_table)
    
    
    conn = sqlite3.connect(":memory:")
    agglo['starttime'] = agglo['Date '] +' '+agglo['Time ']
    agglo['starttime'] = pd.to_datetime(agglo['starttime'])
    agglo['endtime'] = agglo['starttime'] + pd.to_timedelta(agglo['PADDELMIXER RT'], unit='s')
    agglo['BatchNo '] = np.arange(1, len(agglo)+1)
    centerline['datetime'] = centerline['Date ']+' '+centerline['Time ']
    centerline['datetime'] = pd.to_datetime(centerline['datetime'])
    centerline = centerline[centerline['FBC1 BLW HZ']>=0]
    agglo.to_sql('agglo', con=conn, index = False)
    centerline.to_sql('centerline', con=conn, index = False)
    
    qry = "select * from centerline as c inner join agglo as a on c.datetime between a.starttime and a.endtime;"
        
    df1 = pd.read_sql_query(qry, conn)
    
    rm_col = ['Unnamed: 0', 'Date ', 'Time ', 'DateAndTime ', 'datetime', 'Date .1', 'Time .1', 'Shift ']
    
    rem_col = [i for i in rm_col if i in list(df1.columns)]
    
    df1.drop(columns = rem_col, inplace = True)
    df = df1.groupby(['BatchNo ','starttime', 'endtime', 'Recipe ', 'BrandID ', 'BatchID ', 'EQID ']).mean().reset_index()
    
    
    # name = 'merged_data' + '_' + str(int(time.time())) + '.csv'
    # df.to_csv(os.path.join(write_path, name))
    # config['merged_data_path'] = [os.path.join(write_path, name)]
    # write_config(config, config_path)
    append_df_into_table(agglo, database_path, 'Agglmoration')
    append_df_into_table(centerline, database_path, 'Centerline')
    append_df_into_table(df, database_path, 'Merged_Data')
    return df

In [ ]:
def data_preprocessing(database_path, config_path, mode='train'):
    """
    config_path : full path of config file (.../.../xxxxx.txt)
    
    """
    
    config = read_config(config_path)
    
    # merge_data_path = config['merged_data_path'][0]
    # df = pd.read_csv(merge_data_path)
    # write_path = os.path.dirname(merge_data_path)
    table_name = 'Merged_Data'
    if mode=='train':
        df = read_df_table_from_database(database_path, table_name)
    else:
        d = read_df_table_from_database(database_path, table_name)
        kd = d.iloc[-1,:]
        df = pd.DataFrame(np.expand_dims(np.array(kd), 0), columns=list(kd.keys()))
        
    df.columns = [c.strip() for c in df.columns.values.tolist()]
    spcols = [col for col in df.columns if 'SP' in col]

    uwcols = ['DC FINES SP','RM FINES SP']
    spcols = [e for e in spcols if e not in uwcols]
    Totalsp= df[spcols].sum(axis=1)
    df = df.assign(Totalsp = Totalsp.values)

    df = df.drop(df[(df['BrandID'] == 'ARIEL') & (df['POSTMIX CLAY SP'] > 0.0)].index)

    df = df[df['Totalsp'] == 1050]
    
    col = df.columns
    col1 = [a for a in col if '.1' not in a and '.2' not in a]
    df = df.loc[:, col1]
    if mode=='train':
        df = df.iloc[1:, :]

    df.dropna(how = 'all', axis = 1, inplace=True)

    df['starttime'] = pd.to_datetime(df['starttime'])

    df['Month'] = df['starttime'].dt.month

    df['season'] = ['summer' if a==3 or a==4 or a==5 or a==6 else 'monsoon' if a==7 or a==8 or a==9 else 'post_monsoon' if a==10 or a==11 else 'winter' for a in df.Month.values]
    
    id_cols = config['id_columns']

    imp_cols = config['imp_columns']
    
    cols = id_cols+imp_cols
    df.columns = df.columns.str.rstrip()
    df = df.loc[:,cols]

    for c in imp_cols:
        df[c] = df[c].astype(float)

    df.dropna(how = 'any', axis = 0, inplace=True)

    # name = 'preprocessed_data' + '_' +str(int(time.time())) + '.csv'
    # df.to_csv(os.path.join(write_path, name))
    # config['preprocessed_data_path'] = [os.path.join(write_path, name)]
    # write_config(config, config_path)
    
    return df

In [ ]:
def old_method(df, train_data_split, imp_cols, id_cols):
    col_ck = 'Month'
    df.sort_values(by=[col_ck], inplace=True)
    season_list = df[col_ck].unique().tolist()
    
    split = {'train': [], 'test': []}
    for a in season_list:
        temp = df[df[col_ck]==a]
#         temp = shuffle(temp, random_state=0)
        train_n = int(len(temp)*train_data_split)
        split['train'].append(temp.iloc[0:train_n, :])
        split['test'].append(temp.iloc[train_n:, :])
    
    train_df = pd.concat(split['train'])
    test_df = pd.concat(split['test'])
    
    train_cols = list(set(imp_cols) - set(["POSTMIX AC BASE SP"]))
    X_train = train_df.loc[:, train_cols]
    X_test = test_df.loc[:, train_cols]
    
    id_train = train_df.loc[:, id_cols]
    id_test = test_df.loc[:, id_cols]
    
    y_train = train_df["POSTMIX AC BASE SP"].values
    y_test = test_df["POSTMIX AC BASE SP"].values
    return X_train, X_test, y_train, y_test, id_train, id_test

def x_y_scale(df, imp_cols):
    x_col = list(set(imp_cols) - set(["POSTMIX AC BASE SP"]))

    sc_x = preprocessing.StandardScaler().fit(df.loc[:, x_col])
    df.loc[:, x_col] = sc_x.transform(df.loc[:, x_col])

    sc_y = preprocessing.StandardScaler().fit(df.loc[:, ["POSTMIX AC BASE SP"]])
    df.loc[:, ["POSTMIX AC BASE SP"]] = sc_y.transform(df.loc[:, ["POSTMIX AC BASE SP"]])
    
    return df, sc_x, sc_y


def append_df_into_table(df, database_path, table_name):
    
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    # try:
    #     c.execute('CREATE TABLE {} {}'.format(str(table_name), tuple(df.columns)))#Brand text, Price number
    #     conn.commit()
    # except:
    #     pass
    
    df.to_sql(str(table_name), conn, if_exists='append', index = False)
    conn.close()
    return df

def table_to_df(table):
    conn = sqlite3.connect(":memory:")
    c = conn.cursor()
    c.execute('SELECT * FROM {}'.format(table))
    cols=[i[0] for i in c.description]
    d = pd.DataFrame(c.fetchall(), columns=cols)
    return d

def write_table_from_df(df, database_path, table_name):
    
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    try:
        c.execute('CREATE TABLE {} {}'.format(str(table_name), tuple(df.columns)))#Brand text, Price number
        conn.commit()
    except:
        pass
    
    df.to_sql(str(table_name), conn, if_exists='replace', index = False)
    conn.close()
    return df

def read_df_table_from_database(database_path, table_name):
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    c.execute('SELECT * FROM {}'.format(table_name))
    cols=[i[0] for i in c.description]
    d = pd.DataFrame(c.fetchall(), columns=cols)
    return d

In [ ]:
def run_xgboost_with_grid_search(df, brand, database_path, config_path, model_write_path):
    
    """
    df    : preprocessed df
    brand : brand name of the product
    config_path : full path of config file (.../.../xxxxx.txt)
    model_write_path : path where to save the model weights
    
    """
    config = read_config(config_path)
    
    # preprocess_data_path = config['preprocessed_data_path'][0]
    # df = pd.read_csv(preprocess_data_path)
    
    df = df[df['BrandID']==brand]
    
    id_cols = config['id_columns']

    imp_cols = config['imp_columns']
    
    train_data_split = float(config['train_data_split'][0])
    
    cols = id_cols+imp_cols
    df.columns = df.columns.str.rstrip()
    df = df.loc[:,cols]
    
    df, sc_x, sc_y = x_y_scale(df, imp_cols)
    
    X_train, X_test, y_train, y_test, id_train, id_test = old_method(df, train_data_split, imp_cols, id_cols)
    
    regressor = xgb.XGBRegressor()
    
    param_dist = {'n_estimators': list(np.arange(150,1000,100)),
                  'learning_rate': list(np.arange(0.01, 0.5, 0.05)),
                  'max_depth': [3, 4, 5, 6, 7, 8, 9],
                  'reg_lambda':[1],
                  'gamma':[0]
                 }
    
    reg = GridSearchCV(estimator=regressor,  
                       scoring=make_scorer(r2_score),
                        param_grid= param_dist,
                        verbose=False)
    
    reg.fit(X_train, y_train)
    
    y_pred = reg.predict(X_test)
    
    MSE_test = mean_squared_error(y_test, y_pred)
    R2_test = r2_score(y_test, y_pred)
    score = reg.score(X_test, y_test)
    RMSE_test = np.sqrt(MSE_test)
    
    y_pred_tr = reg.predict(X_train)
    MSE_tr = mean_squared_error(y_train, y_pred_tr)
    R2_tr = r2_score(y_train, y_pred_tr)
    score_tr = reg.score(X_train, y_train)
    RMSE_tr = np.sqrt(MSE_tr)
    
    
    if not os.path.exists(model_write_path):
        os.makedirs(model_write_path)
    
    model_name = 'model_'+ str(brand) + '_' + str(time.strftime("%Y-%m-%d")) +'.pickle'
    
    model_file_path = os.path.join(model_write_path, model_name)
    
    f = open(model_file_path, 'wb')
    pickle.dump(reg, f)
    f.close()
    
    X_train.loc[:,:] = sc_x.inverse_transform(X_train.loc[:,:])
    y_train = sc_y.inverse_transform(y_train)
    y_pred_tr = sc_y.inverse_transform(y_pred_tr)
    X_train.loc[:, "POSTMIX AC BASE SP"] = y_train
    X_train.loc[:, "PRED POSTMIX AC BASE SP"] = y_pred_tr
    X_train = pd.concat([X_train, id_train], axis=1)
    
    Features = list(X_test.columns)
    Target_variable = 'POSTMIX AC BASE SP'
    
    X_test.loc[:,:] = sc_x.inverse_transform(X_test.loc[:,:])
    y_test = sc_y.inverse_transform(y_test)
    y_pred = sc_y.inverse_transform(y_pred)
    X_test.loc[:, "POSTMIX AC BASE SP"] = y_test
    X_test.loc[:, "PRED POSTMIX AC BASE SP"] = y_pred
    X_test = pd.concat([X_test, id_test], axis=1)
    
    scaler_name = 'sc_y_'+str(brand) + '_' + str(time.strftime("%Y-%m-%d")) + '.pkl'
    sc_path = os.path.join(model_write_path, scaler_name)
    pickle.dump(sc_y,open(sc_path,'wb'))
    config['gs_sc_y_'+str(brand)+'_path'] = [sc_path]
    
    scaler_x_name = 'sc_x_'+str(brand) + '_' + str(time.strftime("%Y-%m-%d")) + '.pkl'
    sc_x_path = os.path.join(model_write_path, scaler_x_name)
    pickle.dump(sc_x,open(sc_x_path,'wb'))
    config['gs_sc_x_'+str(brand)+'_path'] = [sc_x_path]
    
    best = reg.best_params_
    result_path = os.path.join(model_write_path, 'results_params.csv')

    if not os.path.exists(result_path):
        res = pd.DataFrame(columns=['Date_of_Run', 'BrandID', 'MSE_Train', 'R2_Train',
                                    'MSE_Test', 'R2_Test', 'Gamma', 'Learning_Rate', 'Max_Depth', 'n_estimators', 'Lambda'])
    else:
        res = pd.read_csv(result_path)

    res_data = np.array([time.strftime("%Y-%m-%d %H:%M:%S"), brand, MSE_tr, R2_tr, MSE_test, R2_test,
                best['gamma'], best['learning_rate'], best['max_depth'], best['n_estimators'], best['reg_lambda']])

    temp = pd.DataFrame(np.expand_dims(res_data, 0), columns=['Date_of_Run', 'BrandID', 'MSE_Train', 'R2_Train','MSE_Test','R2_Test',
                                                              'Gamma', 'Learning_Rate', 'Max_Depth', 'n_estimators', 'Lambda'])

    res = pd.concat([res, temp], axis=0, ignore_index=True)
    
    res.loc[0, 'Model_file_location'] = str(model_file_path)
    res.loc[0, 'SC_y_file_location'] = str(sc_path)
    res.loc[0, 'SC_x_file_location'] = str(sc_x_path)
    
    mod_data = np.expand_dims(np.array([brand, str(Features), Target_variable, str(id_cols)]), 0)
    model_df = pd.DataFrame(mod_data, columns=['Brand','Features','Target_variable', 'Id_columns'])

    
    res = append_df_into_table(res, database_path, 'Model_FineTuning_Log')
    
    model_df = append_df_into_table(model_df, database_path, 'Model')
    
    # res.to_csv(result_path)
    
    # total_df = pd.concat([X_train, X_test], axis=0)
    
    # total_df.to_csv(os.path.join(model_write_path, 'full_data_prediction.csv'))
    
    config['gs_model_'+str(brand)+'_path']= [model_file_path]
    # config['results_params_csv_path'] = [result_path]
    # config['full_data_predicton_path'] = [os.path.join(model_write_path, 'full_data_prediction.csv')]
    
    write_config(config, config_path)
    
    return model_df

In [ ]:
def run_xgboost(df, brand, database_path, config_path):
    
    """
    brand : brand name of the product
    config_path : full path of config file (.../.../xxxxx.txt)

    """

    config = read_config(config_path)
    
    # preprocess_data_path = config['preprocessed_data_path'][0]
    # df = pd.read_csv(preprocess_data_path)
    
    df = df[df['BrandID']==brand]
    
    id_cols = config['id_columns']

    imp_cols = config['imp_columns']
    
    train_data_split = float(config['train_data_split'][0])
    
    cols = id_cols+imp_cols
    df.columns = df.columns.str.rstrip()
    df = df.loc[:,cols]
    
    df, sc_x, sc_y = x_y_scale(df, imp_cols)
    
    X_train, X_test, y_train, y_test, id_train, id_test = old_method(df, train_data_split, imp_cols, id_cols)
    
    # params = pd.read_csv(config['results_params_csv_path'][0])
    params = read_df_table_from_database(database_path, 'Model_FineTuning_Log')
    
    params = params[params.BrandID==brand]
    params.sort_values(by=['Date_of_Run'], inplace=True)
    
    reg = xgb.XGBRegressor()
    
    reg.gamma = float(params.Gamma.values[-1])

    reg.learning_rate = float(params.Learning_Rate.values[-1])

    reg.max_depth = int(params.Max_Depth.values[-1])

    reg.n_estimators = int(params.n_estimators.values[-1]);

    reg.reg_lambda = float(params.Lambda.values[-1])
    
    reg.fit(X_train, y_train)
    
    y_pred = reg.predict(X_test)
    
    MSE_test = mean_squared_error(y_test, y_pred)
    R2_test = r2_score(y_test, y_pred)
    score = reg.score(X_test, y_test)
    RMSE_test = np.sqrt(MSE_test)
    
    y_pred_tr = reg.predict(X_train)
    MSE_tr = mean_squared_error(y_train, y_pred_tr)
    R2_tr = r2_score(y_train, y_pred_tr)
    score_tr = reg.score(X_train, y_train)
    RMSE_tr = np.sqrt(MSE_tr)
    
    tomo_date = (dt.datetime.today()).strftime ("%Y-%m-%d")# + dt.timedelta(days=1)
    
    model_write_path = os.path.dirname(params.Model_file_location.values[-1])
    
    daily_model_path = os.path.join(model_write_path, str(tomo_date))
    
    if not os.path.exists(daily_model_path):
        os.makedirs(daily_model_path)
    
    model_name = 'model_'+ str(brand) + '_' + str(tomo_date) +'.pickle'
    
    model_file_path = os.path.join(daily_model_path, model_name)
    
    f = open(model_file_path, 'wb')
    pickle.dump(reg, f)
    f.close()
    
#     X_train.loc[:,:] = sc_x.inverse_transform(X_train.loc[:,:])
#     y_train = sc_y.inverse_transform(y_train)
#     y_pred_tr = sc_y.inverse_transform(y_pred_tr)
#     X_train.loc[:, "POSTMIX AC BASE SP"] = y_train
#     X_train.loc[:, "PRED POSTMIX AC BASE SP"] = y_pred_tr
#     X_train = pd.concat([X_train, id_train], axis=1)
    
#     X_test.loc[:,:] = sc_x.inverse_transform(X_test.loc[:,:])
#     y_test = sc_y.inverse_transform(y_test)
#     y_pred = sc_y.inverse_transform(y_pred)
#     X_test.loc[:, "POSTMIX AC BASE SP"] = y_test
#     X_test.loc[:, "PRED POSTMIX AC BASE SP"] = y_pred
#     X_test = pd.concat([X_test, id_test], axis=1)
    
    scaler_name = 'sc_y_'+str(brand) + '_' + str(tomo_date) + '.pkl'
    sc_path = os.path.join(daily_model_path, scaler_name)
    pickle.dump(sc_y,open(sc_path,'wb'))
    
    scaler_x_name = 'sc_x_'+str(brand) + '_' + str(tomo_date) + '.pkl'
    sc_x_path = os.path.join(daily_model_path, scaler_x_name)
    pickle.dump(sc_x,open(sc_x_path,'wb'))
    
    res_data = np.array([tomo_date, brand, MSE_tr, R2_tr, MSE_test, R2_test,
                params.Gamma.values[-1], params.Learning_Rate.values[-1], params.Max_Depth.values[-1],
                params.n_estimators.values[-1], params.Lambda.values[-1],
                str(model_file_path), str(sc_path), str(sc_x_path)])

    temp = pd.DataFrame(np.expand_dims(res_data, 0), columns=['Date_of_Run', 'BrandID', 'MSE_Train', 'R2_Train','MSE_Test','R2_Test',
                                                              'Gamma', 'Learning_Rate', 'Max_Depth', 'n_estimators', 'Lambda', 'Model_file_location', 'SC_y_file_location', 'SC_x_file_location'])
    
#     res_data = np.array([tomo_date, brand, MSE_tr, R2_tr, MSE_test, R2_test,
#                 params.Gamma.values[-1], params.Learning_Rate.values[-1], params.Max_Depth.values[-1],
#                 params.n_estimators.values[-1], params.Lambda.values[-1],
#                 str(model_file_path), str(sc_path)])

#     temp = pd.DataFrame(np.expand_dims(res_data, 0), columns=['Date_of_Run', 'BrandID', 'MSE_Train', 'R2_Train','MSE_Test','R2_Test',
#                                                               'Gamma', 'Learning_Rate', 'Max_Depth', 'n_estimators', 'Lambda', 'Model_file_location', 'SC_y_file_location'])

    # res = pd.DataFrame(columns=['Date_of_Run', 'BrandID', 'MSE_Train', 'R2_Train',
    #                             'MSE_Test', 'R2_Test', 'Gamma', 'Learning_Rate', 'Max_Depth',
    #                             'n_estimators', 'Lambda', 'Model_file_location', 'SC_y_file_location'])

    temp = append_df_into_table(temp, database_path, 'Model_Training_Log')
    
    # daily_model_list = config['daily_model_list']
    # daily_model_list.append(daily_model_path)
    # config['daily_model_list'] = daily_model_list
    
    # total_df = pd.concat([X_train, X_test], axis=0)
    
    # data_name = 'full_data_prediction_'+ str(tomo_date) + '.csv'
    # total_df.to_csv(os.path.join(daily_model_path, data_name))
        
    write_config(config, config_path)
    
    return temp

In [ ]:
def run_xgboost_prediction(df, database_path, config_path):
    
    """
    df    : preprocessed dataframe
    brand : brand name of the product
    config_path : full path of config file (.../.../xxxxx.txt)
    
    """
    config = read_config(config_path)
    
    # preprocess_data_path = config['preprocessed_data_path'][0]
    # df = pd.read_csv(preprocess_data_path)
    
    brand = df.BrandID.values[0]
    
    df = df[df['BrandID']==brand]
    
    id_cols = config['id_columns']

    imp_cols = config['imp_columns']
    
    train_data_split = float(config['train_data_split'][0])
    
    cols = id_cols+imp_cols
    cols2 = list(set(cols) - set(["POSTMIX AC BASE SP"]))
    df.columns = df.columns.str.rstrip()
    df2 = df.loc[:,cols2]
    
    train_cols = list(set(imp_cols) - set(["POSTMIX AC BASE SP"]))
    
    # if len(config['daily_model_list'])==0:
    #     gs_model_path = config['gs_model_'+str(brand)+'_path']
    #     gs_date = os.path.basename(gs_model_path).split('.')[0].split('_')[-1]
    #     folder_path = os.path.join(os.path.dirname(config['results_params_csv_path'][0]))
    #     model_date = str(gs_date)
    #     pred_write_path = os.path.join(folder_path, str(time.strftime("%Y-%m-%d")))
    #     if not os.path.exists(pred_write_path):
    #         os.makedirs(pred_write_path)
    # else:
    #     folder_path = os.path.abspath(config['daily_model_list'][-1])
    #     model_date = os.path.basename(folder_path)
    #     pred_write_path = str(folder_path)
    
    model_df = read_df_table_from_database(database_path, 'Model_Training_Log')
    model_df = model_df[model_df.BrandID==brand]
    
    if len(model_df)==0:
        model_df = read_df_table_from_database(database_path, 'Model_FineTuning_Log')
        model_df = model_df[model_df.BrandID==brand]
        model_df.sort_values(by=['Date_of_Run'], inplace=True)
    else:
        model_df.sort_values(by=['Date_of_Run'], inplace=True)
    
    folder_path =  os.path.dirname(model_df.Model_file_location.values[-1])
    model_date = os.path.basename(folder_path)
    
    # sc_x = preprocessing.StandardScaler().fit(df2.loc[:, train_cols])
    # df2.loc[:, train_cols] = sc_x.transform(df2.loc[:, train_cols])
    
    # x_scaler = 'sc_x_'+str(brand) + '_' + str(model_date) + '.pkl'
    # sc_x_path = os.path.join(folder_path, x_scaler)
    sc_x_path = model_df.SC_x_file_location.values[-1]
    sc_x = pickle.load(open(os.path.abspath(sc_x_path), 'rb'))
    
    df2.loc[:, train_cols] = sc_x.transform(df2.loc[:, train_cols])
    
    last_model_name = 'model_'+ str(brand) + '_' + str(model_date) +'.pickle'
    last_model_path = os.path.join(folder_path, last_model_name)
    
    model = pickle.load(open(os.path.abspath(last_model_path), 'rb'))
    
    y_pred = model.predict(df2.loc[:, train_cols])
    
    # y_scaler = 'sc_y_'+str(brand) + '_' + str(model_date) + '.pkl'
    # sc_y_path = os.path.join(folder_path, y_scaler)
    sc_y_path = model_df.SC_y_file_location.values[-1]
    sc_y = pickle.load(open(os.path.abspath(sc_y_path), 'rb'))
    
    ori_y_pred = sc_y.inverse_transform(y_pred)
    
#     df.loc[:, "PRED POSTMIX AC BASE SP"] = ori_y_pred
    
    pred_data = np.expand_dims(np.array([df.BatchNo.values[0], df.BrandID.values[0], str(ori_y_pred)]), 0)
    pred_df = pd.DataFrame(pred_data, columns=['BatchNo', 'BrandID', 'Prediction_value'])
    
    pred_df = append_df_into_table(pred_df, database_path, 'Prediction')
    # pred_name = 'prediction' + '_' + str(time.strftime("%Y-%m-%d"))
    
    # df.to_csv(os.path.join(pred_write_path, pred_name+'.csv'))
    
    # config[pred_name] = [os.path.join(pred_write_path, pred_name+'.csv')]
    
    write_config(config, config_path)
    
    return df